In [9]:
# To run the code import these first!!
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps

In [10]:
# Then make connection to the databse
con = ps.connect(dbname="Manshi",user = "postgres",password = "manshi",host = "localhost",port = "5432")

In [8]:
# To see whole data of the table
sql1= """Select * from customer_info"""
df = sqlio.read_sql_query(sql1,con)
df.head()
display(df)

C:\Users\hp\AppData\Local\Temp\ipykernel_13388\1114492077.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query(sql1,con)


,customer_id,first_name,last_name,mobile_number,email,address,total_amount_spent
0,2,Flora,Plants,1290553366,plants@Garden,In every garden,2750.0
1,3,John,Miller,9945126539,johnmiller@gmail.com,"LA Street, USA",95000.0
2,4,Josh,Rusef,8946123094,Josh@mail.com,Let me live somewhere,2300.0
3,5,Harish,Dave,7756439202,dave@gamil.com,"16/A, Shivalik Society",1850.0
4,7,Sanskruti,Tripathi,5769392037,sanskruti@mail.com,"Riverfront, Ahmedabad",360.0
5,10,Roy,Yadav,4589134678,Roy@89.co,"56/I, Complex",675.0
6,11,Jankrut,Dwivedi,9798162749,jankrut@57.com,"70,Madhur Laxmi society",8050.0
7,12,Jack,David,1234567890,J@Dmail.com,"12,Washington",330.0
8,13,Ansh,Ravel,5678903213,Ravel@here.com,"13,Nowwhere",0.0
9,14,Smith,Thomas,1234567890,A@s.co,"12,Somewhere",440.0


In [ ]:
from fpdf import FPDF
from datetime import datetime
import psycopg2
import re

class InvoiceGenerator:
    def __init__(self):
        # Initialize attributes
        self.customer_name = ""
        self.customer_surname = ""
        self.mobile_number = ""
        self.email = ""
        self.address = ""
        self.items = []
        self.pdf = FPDF()

        # Database connection parameters
        self.db_conn_params = {
            'dbname': 'Manshi',
            'user': 'postgres',
            'password': 'manshi',
            'host': 'localhost',
            'port': '5432'
        }

    def save_customer_info_to_database(self, tax_rate=0.1):
        # Method to save customer information to the database
        self.get_items_info()
        try:
            # Connect to the database
            with psycopg2.connect(**self.db_conn_params) as conn:
                with conn.cursor() as cursor:
                    # Calculate the total amount spent by summing the quantity * price for each item
                    total_amount_spent = sum(item['quantity'] * item['price'] for item in self.items)
                    taxes = total_amount_spent * tax_rate

                    # Total
                    total_amount_with_tax = total_amount_spent + taxes

                    # Insert customer information into the database
                    cursor.execute("""
                        INSERT INTO customer_info (first_name, last_name, mobile_number, email, address, total_amount_spent)
                        VALUES (%s, %s, %s, %s, %s, %s)
                        RETURNING customer_id
                    """, (self.customer_name, self.customer_surname, self.mobile_number, self.email, self.address, total_amount_with_tax))
                    # Get the customer_id from the last inserted row
                    self.customer_id = cursor.fetchone()[0]
        except Exception as e:
            print(f"Error saving customer information to the database: {e}")
            
    def search_customer(self):
        # Method to search for a customer by ID
        print("Search Customer:")
        search_criteria = input("Enter customer ID: ")

        try:
            with psycopg2.connect(**self.db_conn_params) as conn:
                with conn.cursor() as cursor:
                        customer_id = int(search_criteria)
                        self.search_by_id(customer_id)
        except Exception as e:
            print(f"Error searching for customer: {e}")
            
    def search_by_id(self, customer_id):
        # Method to search for a customer by their ID in the database
        try:
            with psycopg2.connect(**self.db_conn_params) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        SELECT * FROM customer_info WHERE customer_id = %s
                    """, (customer_id,))
                    result = cursor.fetchone()

                    if result:
                        print("\nCustomer found:")
                        print(f"Customer ID: {result[0]}")
                        print(f"First Name: {result[1]}")
                        print(f"Last Name: {result[2]}")
                        print(f"Mobile Number: {result[3]}")
                        print(f"Email: {result[4]}")
                        print(f"Address: {result[5]}")
                        print(f"Total Amount Spent: {result[6]}")
                    else:
                        print("Customer not found.")
        except Exception as e:
            print(f"Error searching for customer: {e}")    

    def get_customer_info(self):
        # Method to get customer information from the user and save it to the database
        print("Enter Customer Information:")
        self.customer_name = input("First Name: ")
        self.customer_surname = input("Last Name: ")

        # Validate mobile number (exactly 10 digits)
        while True:
            self.mobile_number = input("Mobile Number (10 digits only): ")
            if len(self.mobile_number) == 10 and self.mobile_number.isdigit():
                break
            else:
                print("Invalid mobile number. Please enter exactly 10 digits.")

        # Validate email
        while True:
            self.email = input("Email: ")
            if self.validate_email(self.email):
                break
            else:
                print("Invalid email. Please enter a valid email address.")

        self.address = input("Address: ")

        # Save customer information to the database
        self.save_customer_info_to_database()

    def validate_email(self, email):
        # Method to validate an email address
        match_result = re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email)
        if match_result is not None:
            return True
        else:
            return False

    def get_items_info(self):
        # Method to get item information from the user
        print("\nEnter Item Information (Type 'done' when finished):")
        while True:
            item_name = input("Item Name: ")
            if item_name.lower() == 'done':
                break

            # Validate quantity (should be an integer)
            while True:
                try:
                    quantity = int(input("Quantity: "))
                    if quantity > 0:
                        break
                    else:
                        print("Quantity should be greater than 0.")
                except ValueError:
                    print("Invalid input. Please enter a valid integer for quantity.")

            # Validate price (should be a float or an integer)
            while True:
                try:
                    price = float(input("Price per unit: "))
                    break
                except ValueError:
                    print("Invalid input. Please enter a valid price.")

            self.items.append({'name': item_name, 'quantity': quantity, 'price': price})

    def generate_invoice(self, file_name='invoice.pdf', tax_rate=0.1):
        # Method to generate an invoice and save it as a PDF
        self.pdf.add_page()
        self.pdf.set_font("Arial", size=12)

        # Invoice Table Header
        self.pdf.set_fill_color(215, 173, 237)  # Header fill color
        self.pdf.set_text_color(15, 15, 15)  # Header text color
        self.pdf.set_font("Arial", 'B', 16)  # Bold title font
        self.pdf.cell(190, 20, txt="I n v o i c e", ln=True, align='C', fill=True)
        now = datetime.now()
        date_time = now.strftime("%Y-%m-%d %H:%M:%S")
        self.pdf.set_font("Arial", 'I', 15)  # Italic date font
        self.pdf.cell(190, 10, txt=f"Date: {date_time}", ln=True, align='R')
        self.pdf.set_text_color(0, 0, 0)  # Black text color

        # Customer Information
        self.pdf.ln(10)
        self.pdf.set_text_color(0, 0, 0)  # Text color
        self.pdf.set_font("Arial", size=16)  # Font size
        self.pdf.cell(100, 10, txt=f"Customer: {self.customer_name} {self.customer_surname}", ln=True)
        self.pdf.cell(100, 10, txt=f"Mobile: {self.mobile_number}", ln=True)
        self.pdf.cell(100, 10, txt=f"Email: {self.email}", ln=True)
        self.pdf.cell(100, 10, txt=f"Address: {self.address}", ln=True)

        # Table Header
        self.pdf.ln(15)  # Increased space before the table
        self.pdf.set_fill_color(222, 192, 237)  # Header fill color
        self.pdf.set_text_color(15, 15, 15)  # Table text color

        # Headings
        headings = ["Sr. No.", "Item Name", "Quantity", "Price per unit", "Amount"]
        col_widths = [20, 50, 30, 50, 40, 40, 30]  # Column widths
        for i in range(len(headings)):
            self.pdf.cell(col_widths[i], 15, txt=headings[i], border=1, fill=True, align='C')
        self.pdf.ln()

        # Table Rows
        sr_no = 1
        total_quantity = 0
        total_amount = 0  # Initialize total amount
        for item in self.items:
            self.pdf.cell(col_widths[0], 10, txt=str(sr_no), border="D", align='C')
            self.pdf.cell(col_widths[1], 10, txt=item['name'], border="D", align='C')
            self.pdf.cell(col_widths[2], 10, txt=str(item['quantity']), border="D", align='C')
            self.pdf.cell(col_widths[3], 10, txt=str(item['price']), border="D", align='C')
            amount = item['quantity'] * item['price']
            self.pdf.cell(col_widths[4], 10, txt=str(amount), border="D", align='C', ln=True)
            sr_no += 1
            total_quantity += item['quantity']
            total_amount += amount  # Accumulate total amount
            
        taxes = total_amount * tax_rate

        # Total
        total_amount_with_tax = total_amount + taxes        
        self.pdf.ln(5)
        self.pdf.set_fill_color(230, 211, 240)  # Total row fill color
        self.pdf.cell(col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], 10, txt="Total Quantity", border=1, fill=True, align='C')
        self.pdf.cell(col_widths[4], 10, txt=str(total_quantity), border=1, align='C', ln=True)

        self.pdf.cell(col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], 10, txt="Total Amount", border=1, fill=True, align='C')
        self.pdf.cell(col_widths[4], 10, txt=str(total_amount), border=1, align='C')

        # Taxes
        self.pdf.cell(col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], 10, txt=f"Taxes ({tax_rate*100}%):", border=1, fill=True, align='C')
        self.pdf.cell(col_widths[4], 10, txt=str(taxes), border=1, align='C', ln=True)

        # Total Amount with Taxes
        self.pdf.cell(col_widths[0] + col_widths[1] + col_widths[2] + col_widths[3], 10, txt="Total Amount with Taxes", border=1, fill=True, align='C')
        self.pdf.cell(col_widths[4], 10, txt=str(total_amount_with_tax), border=1, align='C')

        # Save the PDF
        self.pdf.output(file_name)

    def delete_customer(self):
        # Method to delete a customer from the database
        print("Delete Customer:")
        customer_id = input("Enter customer ID to delete: ")

        try:
            with psycopg2.connect(**self.db_conn_params) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        DELETE FROM customer_info WHERE customer_id = %s
                    """, (customer_id,))
                    if cursor.rowcount > 0:
                        print("Customer deleted successfully.")
                        # Display the deleted customer's ID and name
                        print(f"Deleted Customer ID: {customer_id}")
                        self.search_by_id(customer_id)  # Display deleted customer's information
                    else:
                        print("Customer not found.")
        except Exception as e:
            print(f"Error deleting customer: {e}")
            
    def view_all_customers(self):
        # Method to view all customers in the database
        try:
            with psycopg2.connect(**self.db_conn_params) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("SELECT * FROM customer_info")
                    results = cursor.fetchall()
                    if results:
                        print("\nAll Customers:")
                        for result in results:
                            print(f"Customer ID: {result[0]}")
                            print(f"First Name: {result[1]}")
                            print(f"Last Name: {result[2]}")
                            print(f"Mobile Number: {result[3]}")
                            print(f"Email: {result[4]}")
                            print(f"Address: {result[5]}")
                            print(f"Total Amount Spent: {result[6]}")
                            print("-----------------------")
                    else:
                        print("No customers found in the database.")
        except Exception as e:
            print(f"Error viewing all customers: {e}")

invoice_generator = InvoiceGenerator()

while True:
    print("\nMenu:")
    print("1. Enter Customer Details")
    print("2. Search Customer by ID")
    print("3. View All Customers")
    print("4. Delete Customer")
    print("5. Exit")
    choice = input("Enter your choice (1, 2, 3, 4, or 5): ")

    if choice == '1':
        # Get customer information
        invoice_generator.get_customer_info()
        # Generate and save the invoice
        invoice_generator.generate_invoice()
    elif choice == '2':
        # Search for a customer by ID
        invoice_generator.search_customer()
    elif choice == '3':
        # View all customers
        invoice_generator.view_all_customers()
    elif choice == '4':
        # Delete a customer
        invoice_generator.delete_customer()
    elif choice == '5':
        print("Exiting the program.")
        break
    else:
        print("Invalid choice. Please enter 1, 2, 3, 4, or 5.")


Menu:
1. Enter Customer Details
2. Search Customer by ID
3. View All Customers
4. Delete Customer
5. Exit
Enter your choice (1, 2, 3, 4, or 5): 3

All Customers:
Customer ID: 2
First Name: Flora
Last Name: Plants
Mobile Number: 1290553366
Email: plants@Garden
Address: In every garden
Total Amount Spent: 2750.00
-----------------------
Customer ID: 3
First Name: John
Last Name: Miller
Mobile Number: 9945126539
Email: johnmiller@gmail.com
Address: LA Street, USA
Total Amount Spent: 95000.00
-----------------------
Customer ID: 4
First Name: Josh
Last Name: Rusef
Mobile Number: 8946123094
Email: Josh@mail.com
Address: Let me live somewhere
Total Amount Spent: 2300.00
-----------------------
Customer ID: 5
First Name: Harish
Last Name: Dave
Mobile Number: 7756439202
Email: dave@gamil.com
Address: 16/A, Shivalik Society 
Total Amount Spent: 1850.00
-----------------------
Customer ID: 7
First Name: Sanskruti
Last Name: Tripathi
Mobile Number: 5769392037
Email: sanskruti@mail.com
Address: R